# Pairwise Distance: Open-Source XPU Programming with Data Parallel Extensions for Python* language

In [21]:
# SPDX-FileCopyrightText: 2020 - 2023 Intel Corporation
#
# SPDX-License-Identifier: Apache-2.0

In this notebook, learn how to compute pairwise distance, a very popular distance metric applicable in many applications such as machine learning, geospatial, biology, HPC, and so on. 

This tutorial will cover how Data Parallel Extension for NumPy* and Data Parallel Extension for Numba* can be used to compute pairwise distance in NumPy* code efficiently through open-source heterogenous computing and DPC++/SYCL compilation.

## Simple NumPy Script

Here is a simple NumPy* script running pairwise distance on CPU - this is a normal NumPy script without any Data Parallel Extensions for Python* language products included. 

We will also record the time it takes to compute pariwise distance with this script for performance purposes.

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import time

#compute pairwise distance
def pairwise_distance(data, distance):
    data_sqr = np.sum(np.square(data), dtype = np.float32, axis=1)
    np.dot(data, data.T, distance)
    distance *= -2
    np.add(distance, data_sqr.reshape(data_sqr.size, 1), distance)
    np.add(distance, data_sqr, distance)
    np.sqrt(distance, distance)

data = np.random.ranf((10*1024,3)).astype(np.float32)
distance = np.empty(shape=(data.shape[0], data.shape[0]), dtype=np.float32)


#time pairwise distance calculations
start = time.time()
pairwise_distance(data, distance)
print("Time to Compute Pairwise Distance with Stock NumPy on CPU:", time.time() - start)


Time to Compute Pairwise Distance with Stock NumPy on CPU: 2.764047861099243


## Data Parallel Extension for NumPy& Running on Intel® Xeon 4th Generation CPU for Additional Speed-Up

Now let's convert this script into a Data Parallel Extension for NumPy* (dpnp) Script that can be run on any SYCL-supported device (CPU, GPU, etc) by a simple import statement change. 

Instead of `import numpy as np`, let's replace this statement with `import dpnp as np`. That is the only change required!

The script will run on a Intel GPU SYCL-device by default, if available, so we will specify `"cpu"` as the device for the arrays in order to run on a Intel® Xeon 4th Generation CPU. Even though it is running on CPU like NumPy, it has SYCL and oneAPI optimizations underneath it to give it more performance.

In [11]:
#using dpnp to allow heterogenous computing of NumPy!
import dpnp as np 
#import numpy as np

import time

#compute pairwise distance
def pairwise_distance(data, distance):
    data_sqr = np.sum(np.square(data), dtype = np.float32, axis=1) #specify cpu as the device
    np.dot(data, data.T, distance)
    distance *= -2
    np.add(distance, data_sqr.reshape(data_sqr.size, 1), distance)
    np.add(distance, data_sqr, distance)
    np.sqrt(distance, distance)
    
#specify CPU device type to run on CPU
data = np.random.ranf((10*1024,3), device = "cpu").astype(np.float32)
distance = np.empty(shape=(data.shape[0], data.shape[0]), dtype=np.float32, device = "cpu")


#do compilation of kernel first run, we will calculate performance on subsequent runs
pairwise_distance(data, distance)
#time pairwise distance calculations
start = time.time()
pairwise_distance(data, distance)
print("Time to Compute Pairwise Distance with DPNP on CPU:", time.time() - start)

Time to Compute Pairwise Distance with DPNP on CPU: 0.2046055793762207


We already see some speedup just by using DPNP on CPU!

## Data Parallel Extension for NumPy* Running on Intel GPU

Now instead of running this script on Intel CPU, let's run it on a Intel® Data Center GPU Max Series GPU. Since dpnp will use a Intel GPUS SYCL-device by device if it's available, we can remove the device parameter from our calls.

In [14]:
#using dpnp to allow heterogenous computing of NumPy!
import dpnp as np 
#import numpy as np

import time

#compute pairwise distance
def pairwise_distance(data, distance):
    data_sqr = np.sum(np.square(data), dtype = np.float32, axis=1) #specify cpu as the device
    np.dot(data, data.T, distance)
    distance *= -2
    np.add(distance, data_sqr.reshape(data_sqr.size, 1), distance)
    np.add(distance, data_sqr, distance)
    np.sqrt(distance, distance)

#no device specification necessary, by default it will run on Intel GPU SYCL-device if available
data = np.random.ranf((10*1024,3)).astype(np.float32)
distance = np.empty(shape=(data.shape[0], data.shape[0]), dtype=np.float32)


#do compilation of kernel first run, we will calculate performance on subsequent runs
pairwise_distance(data, distance)
#time pairwise distance calculations
start = time.time()
pairwise_distance(data, distance)
print("Time to Compute Pairwise Distance with DPNP on GPU:", time.time() - start)

Time to Compute Pairwise Distance with DPNP on GPU: 0.012220144271850586


Look at that great speedup! All done by a line of code to allow heterogenous computing. 

You can specify the device that you would like to execute on if you have a preferred device. See more in the [dpnp documentation](https://intelpython.github.io/dpnp/).

## Data Parallel Extension for Numba* Enhanced DPNP Script for Further Performance and Compilation on Intel GPU

## Numba Performance Gains Made Easy on any SYCL device

With Data Parallel Extension for Numba* (numba-dpex), you can now run Python code with the Numba-JIT compiler to take advantage of these optimizations on any SYCL device, including Intel GPU! 

Let's try it out with the `@dpjit` function decorator, similar to the Numba `@jit` decorator in allowing JIT compilation but on any SYCL device rather than just CPU.

In [15]:
import dpnp as np
from numba import prange
from numba_dpex import dpjit
import time

@dpjit #using Numba JIT compiler optimizations, now on any SYCL device
def pairwise_distance(data, distance):
    float0 = data.dtype.type(0)

    #prange used for parallel loops for further opt
    for i in prange(data.shape[0]):
        for j in range(data.shape[0]):
            d = float0
            for k in range(data.shape[1]):
                d += (data[i, k] - data[j, k])**2

            distance[j, i] = np.sqrt(d)

data = np.random.ranf((10*1024,3)).astype(np.float32)
distance = np.empty(shape=(data.shape[0], data.shape[0]), dtype=np.float32)

#do compilation first run, we will calculate performance on subsequent runs
#wrapper consistent with previous np and dpnp scripts
pairwise_distance(data, distance)

#time pairwise distance calculations
start = time.time()
pairwise_distance(data, distance)
print("Time to Compute Pairwise Distance with DPNP and Numba-DPEX on GPU:", time.time() - start)

Time to Compute Pairwise Distance with DPNP and Numba-DPEX on GPU: 0.007500171661376953


Wow, even more speedup than in our previous DPNP script from compilation optimizations and SYCL!

### For advanced programmers: SYCL Programming Made Simple by Data Parallel Extension for Numba*

For more advanced programmers, Python developers may want to take advantage of Numba-JIT optimizations and write efficient kernel functions for even more speed-up with  DPNP scripts. 

Now you can write SYCL kernels directly with the `@dpex.kernel` function decorator to use DPC++ SYCL run-time and compilation for near-native speed. No DPC++ SYCL language coding required!

In [16]:
import dpnp as np
import numba_dpex as dpex
import time

# kernel programming similar to Numba's cuda.jit sub-module
# API modeled after SYCL lanaguage and uses the DPC++ run-time
@dpex.kernel
def _pairwise_distance_kernel(data, distance):
    i = dpex.get_global_id(0)
    j = dpex.get_global_id(1)

    data_dims = data.shape[1]

    d = data.dtype.type(0.0)
    for k in range(data_dims):
        tmp = data[i, k] - data[j, k]
        d += tmp * tmp
        
    distance[j, i] = np.sqrt(d)

# for this example, let's put the kernel we just wrote in a wrapper to call it for consistency with our previous scipts
# you can call the kernel directly as well
def pairwise_distance(data, distance):
    _pairwise_distance_kernel[dpex.Range(data.shape[0], data.shape[0])](data, distance)


data = np.random.ranf((10*1024,3)).astype(np.float32)
distance = np.empty(shape=(data.shape[0], data.shape[0]), dtype=np.float32)


#do compilation first run, we will calculate performance on subsequent runs
#wrapper consistent with previous np and dpnp scripts
pairwise_distance(data, distance)

# alternatively, call kernel directly without wrapper:
#_pairwise_distance_kernel[dpex.Range(data.shape[0], data.shape[0])](data, distance)

#time pairwise distance calculations
start = time.time()
pairwise_distance(data, distance)
print("Time to Compute Pairwise Distance with DPNP and Numba-DPEX written SYCL Kernel on GPU:", time.time() - start)

Time to Compute Pairwise Distance with DPNP and Numba-DPEX written SYCL Kernel on GPU: 0.005505800247192383


## For more information on Data Parallel Extensions for Python, please visit:

- [Data Parallel Extensions for Python* Language (dpep) Website](https://intelpython.github.io/DPEP/main/)
- [Data Parallel Extension for NumPy* (dpnp) Documentation](https://intelpython.github.io/dpnp/)
- [Data Parallel Extension for Numba* (numba-dpex) Documentation](https://intelpython.github.io/numba-dpex/latest/index.html)
- [Data Parallel Control (dpctl) Documentation](https://intelpython.github.io/dpctl/latest/index.html)

For support: [Intel Machine Learning and Data Analytics Support Forum](https://community.intel.com/t5/Intel-Distribution-for-Python/bd-p/distribution-python)